In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

C:\Users\hslio\miniforge3\envs\lhs\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 데이터 로드 및 train test로 분할

In [2]:
# df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

In [6]:
df2 = pd.read_csv('38_label_x.csv')

In [3]:
df

title        date  \
0            [제목 : [오아시스]\r\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ]  2020-09-04   
1                         [제목 : [오아시스] 오아시스 매수 했습니다. ]  2021-04-17   
2                         [제목 : [오아시스] 오아시스 매수 했습니다. ]  2021-04-17   
3                         [제목 : [오아시스]\r\n 오아시스 주식이요 ]  2021-04-17   
4                          [제목 : [오아시스] 70만원이면 비싼건가요 ]  2021-04-19   
..                                                 ...         ...   
131                          [제목 : [오아시스] 우크라이나와 러시아 ]  2022-08-31   
132  [제목 : [오아시스] 오늘 날짜(2022.08.31) 더벨 기사,  오아시스 기업...  2022-08-31   
133                 [제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ]  2022-09-02   
134                              [제목 : [오아시스] 계획된 적자 ]  2022-09-05   
135        [제목 : [오아시스] 위기에 재조명받는 ‘오아시스·미트박스’…흑자비결은? ]  2022-09-08   

                                               content  
0    이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...  
1    오아이스 주식 중  pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는...  
2                               얼마에 사셨어요? 저도 관심이 있는데요.  
3    오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...  
4                              30주 70만원에 살려고하는데, 비싼건가요  
..                                                 ...  
131  덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...  
132  코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...  
133  연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...  
134  한때 VC와 스타트업들이 힘을 합쳐 매출액을 늘려 시장 주도권을 가져가기위한 전략으...  
135  오아시스마켓은 올해 상반기 전년 대비 21% 증가한 2024억원 매출액과 171% ...  

[136 rows x 3 columns]

In [8]:
df2.set_index(['date'])

title  \
date                                                            
2020-09-04          [제목 : [오아시스]\r\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ]   
2021-04-17  [제목 : [오아시스] 오아시스 매수 했습니다. ][제목 : [오아시스] 오아시스 ...   
2021-04-19                        [제목 : [오아시스] 70만원이면 비싼건가요 ]   
2021-04-22                        [제목 : [오아시스] 70만원이면 비싼건가요 ]   
2021-04-23  [코멘트 : [오아시스] pe가뭐에요? ][제목 : [오아시스] 망설이다가 못 샀네요 ]   
...                                                       ...   
2022-07-27             [코멘트 : [오아시스] 구라치치말고 아는거 있음 그냥 직접 올려 ]   
2022-08-31  [제목 : [오아시스] 우크라이나와 러시아 ][제목 : [오아시스] 오늘 날짜(20...   
2022-09-02                 [제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ]   
2022-09-05                              [제목 : [오아시스] 계획된 적자 ]   
2022-09-08        [제목 : [오아시스] 위기에 재조명받는 ‘오아시스·미트박스’…흑자비결은? ]   

                                                      content label  
date                                                                 
2020-09-04  이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...     -  
2021-04-17  오아이스 주식 중  pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는...     -  
2021-04-19                          30주 70만원에 살려고하는데, 비싼건가요//     -  
2021-04-22                         어디서 70만원에 살수있어요?나도 사고 싶다//     -  
2021-04-23  pe가뭐에요?//70만원에 살까 말까 고민하고 있는데,팔렸다고 하네요."빨간색매물검...     -  
...                                                       ...   ...  
2022-07-27                           구라치치말고 아는거 있음 그냥 직접 올려//     -  
2022-08-31  덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...     -  
2022-09-02  연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...     -  
2022-09-05  한때 VC와 스타트업들이 힘을 합쳐 매출액을 늘려 시장 주도권을 가져가기위한 전략으...     -  
2022-09-08  오아시스마켓은 올해 상반기 전년 대비 21% 증가한 2024억원 매출액과 171% ...     -  

[86 rows x 3 columns]

## null 값 확인

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  133 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


In [5]:
# NULL 값 있는 행 확인
df[df['content'].isnull()]

title        date content
31              [제목 : [오아시스] 상장노력은 하는거죠? ]  2021-07-09     NaN
40                [제목 : [오아시스] 조용히 갑시다~~ ]  2021-07-16     NaN
43  [제목 : [오아시스] 지금 잘가고 있긴 한데 스피드 좀 올려~~ ]  2021-07-29     NaN

In [6]:
# 공백으로 NULL 값 채우기
df = df.fillna('')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  136 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


내용에 빈 값이 있지만 title에서 추출할 것이 있지 않을까 해서 공백으로 채워줌

## index 날짜로 바꿔주기

In [8]:
# date를 오름차순 정렬
df.date = df.date.sort_index(ascending=True)
# index로 설정
df = df.set_index('date')
df

title  \
date                                                            
2020-09-04            [제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ]   
2021-04-17                       [제목 : [오아시스] 오아시스 매수 했습니다. ]   
2021-04-17                       [제목 : [오아시스] 오아시스 매수 했습니다. ]   
2021-04-17                         [제목 : [오아시스]\n 오아시스 주식이요 ]   
2021-04-19                        [제목 : [오아시스] 70만원이면 비싼건가요 ]   
...                                                       ...   
2022-08-31                          [제목 : [오아시스] 우크라이나와 러시아 ]   
2022-08-31  [제목 : [오아시스] 오늘 날짜(2022.08.31) 더벨 기사,  오아시스 기업...   
2022-09-02                 [제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ]   
2022-09-05                              [제목 : [오아시스] 계획된 적자 ]   
2022-09-08        [제목 : [오아시스] 위기에 재조명받는 ‘오아시스·미트박스’…흑자비결은? ]   

                                                      content  
date                                                           
2020-09-04  이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...  
2021-04-17  오아이스 주식 중  pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는...  
2021-04-17                             얼마에 사셨어요? 저도 관심이 있는데요.  
2021-04-17  오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...  
2021-04-19                            30주 70만원에 살려고하는데, 비싼건가요  
...                                                       ...  
2022-08-31  덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...  
2022-08-31  코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...  
2022-09-02  연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...  
2022-09-05  한때 VC와 스타트업들이 힘을 합쳐 매출액을 늘려 시장 주도권을 가져가기위한 전략으...  
2022-09-08  오아시스마켓은 올해 상반기 전년 대비 21% 증가한 2024억원 매출액과 171% ...  

[136 rows x 2 columns]

## 특수문자 제거 정규표현식

In [9]:
df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_1489/1046961940.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
/var/folders/1d/lfkpl6ps02xgh0rdprtbmgvw0000gn/T/ipykernel_1489/1046961940.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')


In [10]:
df

title  \
date                                                            
2020-09-04                 제목  오아시스\n 팬데믹시대에 내마음의 표준을 바꾸는 유통    
2021-04-17                             제목  오아시스 오아시스 매수 했습니다    
2021-04-17                             제목  오아시스 오아시스 매수 했습니다    
2021-04-17                              제목  오아시스\n 오아시스 주식이요    
2021-04-19                             제목  오아시스 70만원이면 비싼건가요    
...                                                       ...   
2022-08-31                               제목  오아시스 우크라이나와 러시아    
2022-08-31  제목  오아시스 오늘 날짜20220831 더벨 기사  오아시스 기업가치 최대 2조 제시    
2022-09-02                      제목  오아시스 이 시기에 오히려 주목해야 할 기업    
2022-09-05                                   제목  오아시스 계획된 적자    
2022-09-08                  제목  오아시스 위기에 재조명받는 오아시스미트박스흑자비결은    

                                                      content  
date                                                           
2020-09-04  이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다매장에서 인테넷의...  
2021-04-17  오아이스 주식 중  pe가 보유한 물량이 시장에서 움직이는 듯 하네요어제 아시는 분...  
2021-04-17                               얼마에 사셨어요 저도 관심이 있는데요  
2021-04-17     오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면 팔육일팔 삼육사일로연락주세요  
2021-04-19                             30주 70만원에 살려고하는데 비싼건가요  
...                                                       ...  
2022-08-31  덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는 길고 짧은건 대봐야 안다고우크...  
2022-08-31  코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...  
2022-09-02  연일 금리가 급등하고 있다 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유니...  
2022-09-05  한때 VC와 스타트업들이 힘을 합쳐 매출액을 늘려 시장 주도권을 가져가기위한 전략으...  
2022-09-08  오아시스마켓은 올해 상반기 전년 대비 21 증가한 2024억원 매출액과 171 증가...  

[136 rows x 2 columns]

## 토큰화

In [11]:
okt = Okt()


### 추출했던 word_list에서 불용어들만 정리

In [14]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [15]:
with open('stopword_add_title.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    data = list(reader)

In [16]:
data

[['오아시스',
  '내',
  '을',
  '주',
  '식이',
  '요',
  '이면',
  'pe',
  '뭐',
  '에요',
  '위',
  '글',
  '기',
  '때',
  '옆',
  '집',
  '그렇게',
  '일',
  '그',
  '놈',
  '함',
  '알다',
  '1',
  '71',
  '년',
  '안',
  '영',
  '억',
  '임',
  '네',
  '실',
  '137만',
  '가격',
  '다르다',
  '나',
  'K',
  '삼촌',
  '있다',
  '한데',
  '200',
  '얘기',
  '14677',
  '로',
  '이다',
  '에서',
  '입니다',
  '팔',
  '라고',
  '4',
  '대',
  '시',
  '총',
  '평',
  '어떻다',
  '전',
  '3000000',
  '후',
  '액',
  '분과',
  '신',
  '적',
  '주네',
  '수',
  '드리다',
  '요나',
  '분',
  '성은',
  '5',
  '백원',
  '주식',
  '수가',
  '하',
  '지',
  '바로',
  '몇',
  '일이',
  '죠',
  '향',
  '다',
  '업자',
  '만',
  '뭔',
  '건가',
  '기업',
  '벨',
  '류',
  '된거',
  '치다',
  '이상',
  '누구',
  '이름',
  '인가',
  '아시',
  '부터',
  '작',
  '공',
  '모',
  '가가',
  '장',
  '극',
  '40',
  '시간',
  '2',
  '지다',
  '50분',
  '내용',
  '50',
  '배',
  '써다',
  '29일',
  '까지',
  '날짜',
  '4월',
  '저',
  '3',
  '관련',
  '기준',
  '및',
  '52일',
  '조',
  '라',
  '쓰다',
  '어디',
  '어나',
  '오는',
  '즈',
  '아',
  '성',
  '순',
  '익',
  '중',

In [17]:
stopword_add = sum(data, [])
stopword_add

['오아시스',
 '내',
 '을',
 '주',
 '식이',
 '요',
 '이면',
 'pe',
 '뭐',
 '에요',
 '위',
 '글',
 '기',
 '때',
 '옆',
 '집',
 '그렇게',
 '일',
 '그',
 '놈',
 '함',
 '알다',
 '1',
 '71',
 '년',
 '안',
 '영',
 '억',
 '임',
 '네',
 '실',
 '137만',
 '가격',
 '다르다',
 '나',
 'K',
 '삼촌',
 '있다',
 '한데',
 '200',
 '얘기',
 '14677',
 '로',
 '이다',
 '에서',
 '입니다',
 '팔',
 '라고',
 '4',
 '대',
 '시',
 '총',
 '평',
 '어떻다',
 '전',
 '3000000',
 '후',
 '액',
 '분과',
 '신',
 '적',
 '주네',
 '수',
 '드리다',
 '요나',
 '분',
 '성은',
 '5',
 '백원',
 '주식',
 '수가',
 '하',
 '지',
 '바로',
 '몇',
 '일이',
 '죠',
 '향',
 '다',
 '업자',
 '만',
 '뭔',
 '건가',
 '기업',
 '벨',
 '류',
 '된거',
 '치다',
 '이상',
 '누구',
 '이름',
 '인가',
 '아시',
 '부터',
 '작',
 '공',
 '모',
 '가가',
 '장',
 '극',
 '40',
 '시간',
 '2',
 '지다',
 '50분',
 '내용',
 '50',
 '배',
 '써다',
 '29일',
 '까지',
 '날짜',
 '4월',
 '저',
 '3',
 '관련',
 '기준',
 '및',
 '52일',
 '조',
 '라',
 '쓰다',
 '어디',
 '어나',
 '오는',
 '즈',
 '아',
 '성',
 '순',
 '익',
 '중',
 '인데',
 '20220831']

In [42]:
# 불용어 처리 

stopwords = stopword_add + ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다',
                            '\n','\n ','제목','\xa0','오','\xa0\xa0','X','\xa0','오','되다','보다','않다',
                           '것']
stopwords

['오아시스',
 '내',
 '을',
 '주',
 '식이',
 '요',
 '이면',
 'pe',
 '뭐',
 '에요',
 '위',
 '글',
 '기',
 '때',
 '옆',
 '집',
 '그렇게',
 '일',
 '그',
 '놈',
 '함',
 '알다',
 '1',
 '71',
 '년',
 '안',
 '영',
 '억',
 '임',
 '네',
 '실',
 '137만',
 '가격',
 '다르다',
 '나',
 'K',
 '삼촌',
 '있다',
 '한데',
 '200',
 '얘기',
 '14677',
 '로',
 '이다',
 '에서',
 '입니다',
 '팔',
 '라고',
 '4',
 '대',
 '시',
 '총',
 '평',
 '어떻다',
 '전',
 '3000000',
 '후',
 '액',
 '분과',
 '신',
 '적',
 '주네',
 '수',
 '드리다',
 '요나',
 '분',
 '성은',
 '5',
 '백원',
 '주식',
 '수가',
 '하',
 '지',
 '바로',
 '몇',
 '일이',
 '죠',
 '향',
 '다',
 '업자',
 '만',
 '뭔',
 '건가',
 '기업',
 '벨',
 '류',
 '된거',
 '치다',
 '이상',
 '누구',
 '이름',
 '인가',
 '아시',
 '부터',
 '작',
 '공',
 '모',
 '가가',
 '장',
 '극',
 '40',
 '시간',
 '2',
 '지다',
 '50분',
 '내용',
 '50',
 '배',
 '써다',
 '29일',
 '까지',
 '날짜',
 '4월',
 '저',
 '3',
 '관련',
 '기준',
 '및',
 '52일',
 '조',
 '라',
 '쓰다',
 '어디',
 '어나',
 '오는',
 '즈',
 '아',
 '성',
 '순',
 '익',
 '중',
 '인데',
 '20220831',
 '의',
 '가',
 '이',
 '은',
 '들',
 '는',
 '좀',
 '잘',
 '걍',
 '과',
 '도',
 '를',
 '으로',
 '자',
 '에',
 '와',
 '한',
 '하다',


In [43]:
content = []
for sentence in tqdm(df['content']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_content = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    content.append(stopwords_removed_content)

100%|██████████████████████████████████████████████████████████████████| 136/136 [00:00<00:00, 503.42it/s]


In [44]:
print(len(content))

136


In [45]:
from collections import Counter

In [46]:
# 한 날짜? 한 행당 횟수 추출

for i in range(136):
    vocab = Counter(content[i])
    print(vocab)


Counter({'유통': 2, '변화': 2, '힘': 2, '이번': 1, '코로나': 1, '시장': 1, '판도': 1, '5년': 1, '앞당겨지다': 1, '가다': 1, '오다': 1, '매장': 1, '인테넷': 1, '제대로': 1, '시키다': 1, '선진국': 1, '사재기': 1, '우리나라': 1, '경우': 1, '배달민족': 1, '후손': 1, '이라서': 1, '그렇다': 1, '생필품': 1, '전혀': 1, '문제': 1, '없다': 1, '오히려': 1, '폭발': 1, '물량': 1, '증가': 1, '눈': 1, '보이다': 1, '온라인': 1, '점점': 1, '거세지다': 1, '오프라인': 1, '시대': 1, '저물다': 1, '시점': 1, '미리': 1, '선점': 1, '자다': 1, '업체': 1, '인': 1, '앞': 1, '대박': 1, '날': 1, '이여': 1})
Counter({'성장': 3, '성남': 2, '아이스': 1, '가다': 1, '보유': 1, '물량': 1, '시장': 1, '움직이다': 1, '듯': 1, '어제': 1, '함께': 1, '조금': 1, '담다': 1, '마켓': 1, '컬리': 1, '비다': 1, '떨어지다': 1, '거래': 1, '섹터': 1, '작다': 1, '아쉽다': 1, '부분': 1, '수혈': 1, '받다': 1, '자금': 1, '가지': 1, '고': 1, '견실': 1, '기대하다': 1, '회사': 1, '창문': 1, '넘다': 1, '공단': 1, '물류': 1, '센터': 1, '보이다': 1, '같다': 1, '산업': 1, '단지': 1, '도합': 1, '니': 1})
Counter({'얼마': 1, '사다': 1, '관심': 1})
Counter({'사다': 1, '게시판': 1, '올리다': 1, '파다': 1, '계시': 1, '면': 1, '팔육일팔': 1, '삼육사일': 1, '연락': 1, '줄다': 1})
Cou

In [47]:
# 전체 중 빈도수 출력

all_word_list_content = sum(content, [])
vocab = Counter(all_word_list_content)
print(vocab)

Counter({'상장': 38, '투자': 21, '매출': 17, '없다': 14, '마켓': 14, '오다': 13, '인': 13, '거래': 13, '고': 13, '가치': 13, '청구': 13, '컬리': 12, '하고': 12, '올해': 12, '이익': 12, '시장': 11, '센터': 11, '영업': 11, '성장': 10, '회사': 10, '올리다': 10, '지금': 10, '내년': 10, '액면': 10, '분할': 10, '1조': 10, '약': 10, '증권': 10, '받다': 9, '넘다': 9, '물류': 9, '같다': 9, '겁니다': 9, '적자': 9, '주다': 9, '등': 8, '공모': 8, '말': 8, '가다': 7, '물량': 7, '듯': 7, '매도': 7, '많이': 7, '되어다': 7, '흑자': 7, '업계': 7, '진행': 7, '면': 6, '싶다': 6, '액면가': 6, '5000원': 6, '정도': 6, '계획': 6, '많다': 6, '아니다': 6, '100원': 6, '최근': 6, '실적': 6, '발행': 6, '작업': 6, '사': 6, '평가': 6, '해당': 6, '높다': 6, '그렇다': 5, '보이다': 5, '자다': 5, '날': 5, '사다': 5, '비싸다': 5, '새벽': 5, '배송': 5, '플랫폼': 5, '나오다': 5, '하반기': 5, '내다': 5, '못': 5, '더': 5, '다른': 5, '상승': 5, '먹다': 5, '밸류': 5, '들어오다': 5, '신청': 5, '확인': 5, '답변': 5, '목표': 5, '시기': 5, '상반기': 5, '2조': 5, '수준': 5, '한국': 5, '심사': 5, '서': 5, '기업가': 5, '금리': 5, '증가': 4, '관심': 4, '나다': 4, '매물': 4, '밖에': 4, '너무': 4, '싸다': 4, '기관': 4, '매수': 4, '님': 4, '조

In [48]:
word_list = list(vocab.keys())
word_list

['이번',
 '코로나',
 '유통',
 '시장',
 '판도',
 '5년',
 '앞당겨지다',
 '변화',
 '가다',
 '오다',
 '매장',
 '인테넷',
 '힘',
 '제대로',
 '시키다',
 '선진국',
 '사재기',
 '우리나라',
 '경우',
 '배달민족',
 '후손',
 '이라서',
 '그렇다',
 '생필품',
 '전혀',
 '문제',
 '없다',
 '오히려',
 '폭발',
 '물량',
 '증가',
 '눈',
 '보이다',
 '온라인',
 '점점',
 '거세지다',
 '오프라인',
 '시대',
 '저물다',
 '시점',
 '미리',
 '선점',
 '자다',
 '업체',
 '인',
 '앞',
 '대박',
 '날',
 '이여',
 '아이스',
 '보유',
 '움직이다',
 '듯',
 '어제',
 '함께',
 '조금',
 '담다',
 '마켓',
 '컬리',
 '비다',
 '성장',
 '떨어지다',
 '거래',
 '섹터',
 '작다',
 '아쉽다',
 '부분',
 '수혈',
 '받다',
 '자금',
 '가지',
 '고',
 '견실',
 '기대하다',
 '회사',
 '창문',
 '넘다',
 '성남',
 '공단',
 '물류',
 '센터',
 '같다',
 '산업',
 '단지',
 '도합',
 '니',
 '얼마',
 '사다',
 '관심',
 '게시판',
 '올리다',
 '파다',
 '계시',
 '면',
 '팔육일팔',
 '삼육사일',
 '연락',
 '줄다',
 '30',
 '70만원',
 '살리다',
 '비싸다',
 '가요',
 '어디서',
 '살다',
 '나다',
 '사고',
 '싶다',
 '말다',
 '고민',
 '하고',
 '팔리다',
 '빨간색',
 '매물',
 '검색',
 '인천',
 '딜러',
 '매도',
 '지금',
 'seoul',
 '거래소',
 '밖에',
 '고고',
 '750000원',
 '너무',
 '액면가',
 '5000원',
 '이라고는',
 '하지만',
 '총각',
 '오픈마켓',
 '버전',
 '정도',
 '비교',
 '많이',
 

In [49]:
word_rank = list(vocab.values())
word_rank

[2,
 1,
 3,
 11,
 1,
 1,
 1,
 2,
 7,
 13,
 2,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 1,
 3,
 5,
 1,
 2,
 3,
 14,
 1,
 3,
 7,
 4,
 1,
 5,
 3,
 1,
 1,
 3,
 1,
 1,
 1,
 2,
 1,
 5,
 1,
 13,
 3,
 2,
 5,
 1,
 1,
 2,
 1,
 7,
 2,
 1,
 1,
 3,
 14,
 12,
 2,
 10,
 1,
 13,
 1,
 1,
 1,
 2,
 1,
 9,
 2,
 3,
 13,
 1,
 2,
 10,
 1,
 9,
 2,
 1,
 9,
 11,
 9,
 1,
 1,
 1,
 2,
 1,
 5,
 4,
 1,
 10,
 3,
 2,
 6,
 1,
 1,
 1,
 2,
 1,
 3,
 3,
 5,
 2,
 2,
 2,
 4,
 1,
 6,
 1,
 1,
 12,
 1,
 1,
 4,
 1,
 1,
 2,
 7,
 10,
 1,
 3,
 4,
 1,
 1,
 4,
 6,
 6,
 1,
 2,
 1,
 1,
 1,
 6,
 1,
 7,
 1,
 1,
 4,
 9,
 9,
 2,
 5,
 5,
 1,
 1,
 1,
 38,
 6,
 3,
 2,
 1,
 3,
 1,
 2,
 1,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 3,
 4,
 1,
 6,
 1,
 1,
 2,
 3,
 2,
 2,
 1,
 1,
 1,
 3,
 4,
 1,
 1,
 2,
 1,
 1,
 1,
 9,
 1,
 3,
 1,
 3,
 2,
 1,
 1,
 4,
 1,
 2,
 6,
 6,
 2,
 5,
 3,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 6,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 3,
 2,
 2,
 7,
 2,
 5,
 1,
 1,
 1,
 1,
 3,
 4,
 1,
 1,
 1,
 2,
 3,
 

In [50]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

word  count
0       이번      2
1      코로나      1
2       유통      3
3       시장     11
4       판도      1
...    ...    ...
1197    에는      1
1198    치의      1
1199  프리미엄      1
1200    요인      1
1201    축소      1

[1202 rows x 2 columns]

In [51]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [40]:
for i in range(26):
    print(word_data[ i*10 : (i+1)*10 ])

  word  count
0   상장     38
1   투자     21
2    것     20
3   매출     17
4   마켓     14
5   없다     14
6   가치     13
7    고     13
8   거래     13
9    인     13
   word  count
10   청구     13
11   오다     13
12   컬리     12
13   이익     12
14   하고     12
15   올해     12
16   시장     11
17   영업     11
18   센터     11
19   1조     10
   word  count
20   성장     10
21   증권     10
22    약     10
23   회사     10
24   분할     10
25   액면     10
26   내년     10
27  올리다     10
28   지금     10
29   받다      9
   word  count
30   물류      9
31   넘다      9
32   적자      9
33  겁니다      9
34   같다      9
35   주다      9
36   공모      8
37    등      8
38    말      8
39    듯      7
     word  count
40     물량      7
41     가다      7
42     많이      7
43    되어다      7
44     매도      7
45     업계      7
46     진행      7
47     흑자      7
48  5000원      6
49     많다      6
   word  count
50   정도      6
51  액면가      6
52   최근      6
53    사      6
54   계획      6
55  아니다      6
56   작업      6
57   싶다      6
58   높다      6
59   발행      6

word_data.to_csv('sentimental(38).csv')

In [52]:
word_data

word  count
0      상장     38
1      투자     21
2      매출     17
3      마켓     14
4      없다     14
...   ...    ...
1197   필연      1
1198   만년      1
1199  테슬라      1
1200   요건      1
1201   축소      1

[1202 rows x 2 columns]

In [60]:
word_data.to_csv('community_content.csv')

sentimental = pd.read_csv('sentimental(38).csv').drop(columns=['Unnamed: 0'])
sentimental = sentimental.fillna('-')

sentimental

sentimental.to_csv('sentimental(38).csv')

In [94]:
community_title_label = pd.read_csv('sentimental(38)_label_title.csv').drop(columns=['Unnamed: 0'])
community_title_label

word  count label
0    코멘트     32    --
1     상장     12     1
2     되다      9     1
3     1조      8     1
4     청구      8     1
..   ...    ...   ...
250   같다      1     0
251    상      1    --
252  장하다      1    --
253   최소      1     0
254   비결      1     0

[255 rows x 3 columns]

### labeling 한 거에서 추가 불용어 처리

In [97]:
community_title_label = community_title_label[~community_title_label['label'].str.contains('--')]
community_title_label

word  count label
1      상장     12     1
2      되다      9     1
3      1조      8     1
4      청구      8     1
5      액면      7     0
..    ...    ...   ...
248     앞      1     0
249  좋아지다      1     0
250    같다      1     0
253    최소      1     0
254    비결      1     0

[232 rows x 3 columns]

In [98]:
community_title_label.to_csv('community_title_label.csv')